In [1]:
import json
import pandas as pd
# from census_helper import Census
import requests
import numpy as np
import us
import census
import re
import os
import logging
from typing import List

In [2]:
with open('acs5_variables_2018.json', 'r') as file:
    data = file.read()

In [3]:
parsed_data = json.loads(data)

# Extract the group names
group_names = list(parsed_data["variables"].keys())
filtered_variable_names = [name for name in group_names if re.match(r'[BC]\d', name)]

In [4]:
# filtered_variable_names.sort()
len(filtered_variable_names)

26996

In [17]:
class Census:
    '''
    Note from Andrew: This class is taken directly from the synthpop package.
    It was originally imported using:
    from synthpop.census_helpers import Census
    I don't really know much about this otherwise, other than that it works for querying ACS data.
    You can probably query ACS data through a different method too, should this method from SynthPop stop working
    Downloading PUMS data through this doesn't work through this, because it doesn't provide the serial number which we need
    '''

    def __init__(self, key):
        self.c = census.Census(key)
        self.base_url = "https://s3-us-west-1.amazonaws.com/synthpop-data2/"
        self.pums_relationship_file_url = self.base_url + "tract10_to_puma.csv"
        self.pums_relationship_df = None
        self.pums10_population_base_url = \
            self.base_url + "puma10_p_%s_%s.csv"
        self.pums10_household_base_url = \
            self.base_url + "puma10_h_%s_%s.csv"
        self.pums00_population_base_url = \
            self.base_url + "puma00_p_%s_%s.csv"
        self.pums00_household_base_url = \
            self.base_url + "puma00_h_%s_%s.csv"
        self.pums_population_state_base_url = \
            self.base_url + "puma_p_%s.csv"
        self.pums_household_state_base_url = \
            self.base_url + "puma_h_%s.csv"
        self.fips_url = self.base_url + "national_county.txt"
        self.fips_df = None
        self.pums_cache = {}

    # df1 is the disaggregate data frame (e.g. block groups)
    # df2 is the aggregate data frame (e.g. tracts)
    # need to scale down df2 variables to df1 level totals
    def _scale_and_merge(self, df1, tot1, df2, tot2, columns_to_scale,
                         merge_columns, suffixes):
        df = pd.merge(df1, df2, left_on=merge_columns, right_on=merge_columns,
                      suffixes=suffixes)

        # going to scale these too so store current values
        tot2, tot1 = df[tot2], df[tot1]
        # if agg number if 0, disaggregate should be 0
        # note this is filled by fillna below
        assert np.all(tot1[tot2 == 0] == 0)

        for col in columns_to_scale:
            df[col] = df[col] / tot2 * tot1
            # round?
            df[col] = df[col].fillna(0).astype('int')
        return df

    def block_group_query(self, census_columns, state, county, year=2016,
                          tract=None, id=None):
        if id is None:
            id = "*"
        return self._query(census_columns, state, county,
                           forstr="block group:%s" % id,
                           tract=tract, year=year)

    def tract_query(self, census_columns, state, county, year=2016,
                    tract=None):
        if tract is None:
            tract = "*"
        return self._query(census_columns, state, county,
                           forstr="tract:%s" % tract,
                           year=year)

    def _query(self, census_columns, state, county, forstr,
               year, tract=None):
        c = self.c

        state, county = self.try_fips_lookup(state, county)

        if tract is None:
            in_str = 'state:%s county:%s' % (state, county)
        else:
            in_str = 'state:%s county:%s tract:%s' % (state, county, tract)

        dfs = []

        # unfortunately the api only queries 50 columns at a time
        # leave room for a few extra id columns
        def chunks(l, n):
            """ Yield successive n-sized chunks from l.
            """
            for i in range(0, len(l), n):
                yield l[i:i + n]

        for census_column_batch in chunks(census_columns, 45):
            census_column_batch = list(census_column_batch)
            # print(census_column_batch)
            d = c.acs5.get(['NAME'] + census_column_batch,
                           geo={'for': forstr,
                                'in': in_str}, year=year)
            # print(d)
            df = pd.DataFrame(d)
            # df[census_column_batch] = df[census_column_batch].astype('int') # This is the original, however it runs into problems if the value is None or NA
            for name in census_column_batch: # This is the new added for loop as a replacement for the previous line
                if df[name].dtype == 'int64':
                    df[name] = df[name].astype('int')
            dfs.append(df)

        assert len(dfs) >= 1
        df = dfs[0]
        for mdf in dfs[1:]:
            df = pd.merge(df, mdf, on="NAME", suffixes=("", "_ignore"))
            drop_cols = list(filter(lambda x: "_ignore" in x, df.columns))
            df = df.drop(drop_cols, axis=1)

        return df

    def block_group_and_tract_query(self, block_group_columns,
                                    tract_columns, state, county,
                                    merge_columns, block_group_size_attr,
                                    tract_size_attr, year=2016, tract=None):
        df2 = self.tract_query(tract_columns, state, county, tract=tract,
                               year=year)
        df1 = self.block_group_query(block_group_columns, state, county,
                                     tract=tract, year=year)

        df = self._scale_and_merge(df1, block_group_size_attr, df2,
                                   tract_size_attr, tract_columns,
                                   merge_columns, suffixes=("", "_ignore"))
        drop_cols = list(filter(lambda x: "_ignore" in x, df.columns))
        df = df.drop(drop_cols, axis=1)

        return df

    def _get_pums_relationship(self):
        if self.pums_relationship_df is None:
            self.pums_relationship_df = \
                pd.read_csv(self.pums_relationship_file_url, dtype={
                    "statefp": "object",
                    "countyfp": "object",
                    "tractce": "object",
                    "puma10_id": "object",
                    "puma00_id": "object",
                })
        return self.pums_relationship_df

    def _get_fips_lookup(self):
        if self.fips_df is None:
            self.fips_df = pd.read_csv(
                self.fips_url,
                dtype={
                    "State ANSI": "object",
                    "County ANSI": "object"
                },
                index_col=["State",
                           "County Name"]
            )
            del self.fips_df["ANSI Cl"]
        return self.fips_df

    def tract_to_puma(self, state, county, tract):

        state, county = self.try_fips_lookup(state, county)

        df = self._get_pums_relationship()
        q = "statefp == '%s' and countyfp == '%s' and tractce == '%s'" % (
            state, county, tract)
        r = df.query(q)
        return r["puma10_id"].values[0], r["puma00_id"].values[0]

    def _read_csv(self, loc, **kargs):
        if loc not in self.pums_cache:
            pums_df = pd.read_csv(loc, dtype={
                "PUMA10": "object",
                "PUMA00": "object",
                "ST": "object"
            }, **kargs)
            pums_df = pums_df.rename(columns={
                'PUMA10': 'puma10',
                'PUMA00': 'puma00',
                'SERIALNO': 'serialno'
            })
            self.pums_cache[loc] = pums_df
        return self.pums_cache[loc]

    def download_population_pums(self, state, puma10=None, puma00=None,
                                 **kargs):
        state = self.try_fips_lookup(state)
        if (puma10 is None) & (puma00 is None):
            return self._read_csv(self.pums_population_state_base_url % (state),
                                  **kargs)
        pums = self._read_csv(self.pums10_population_base_url % (state, puma10),
                              **kargs)
        if puma00 is not None:
            pums00 = self._read_csv(
                self.pums00_population_base_url % (state, puma00), **kargs)
            pums = pd.concat([pums, pums00], ignore_index=True)
        return pums

    def download_household_pums(self, state, puma10=None, puma00=None, **kargs):
        state = self.try_fips_lookup(state)
        if (puma10 is None) & (puma00 is None):
            return self._read_csv(self.pums_household_state_base_url % (state),
                                  **kargs)
        pums = self._read_csv(self.pums10_household_base_url % (state, puma10),
                              **kargs)
        if puma00 is not None:
            pums00 = self._read_csv(
                self.pums00_household_base_url % (state, puma00), **kargs)
            pums = pd.concat([pums, pums00], ignore_index=True)

        # filter out gq and empty units (non-hh records)
        pums = pums[(pums.RT == 'H') & (pums.NP > 0) & (pums.TYPE == 1)]

        return pums

    def try_fips_lookup(self, state, county=None):
        df = self._get_fips_lookup()

        if county is None:
            try:
                return getattr(us.states, state).fips
            except:
                pass
            return state

        try:
            return df.loc[(state, county)]
        except:
            pass
        return state, county

# import time
# c = Census("285e70bec59918991430e98163a4cda2802b4f01")
#
# income_columns = ['B19001_0%02dE' % i for i in range(1, 18)]
# hhsize_columns = ['B11016_0%02dE' % i for i in range(1, 17)]
# hhrace_columns = ['B25006_0%02dE' % i for i in range(1, 11)]
# hhown_columns = ['B25003_0%02dE' % i for i in range(1, 4)]
# work_columns = ['B08202_0%02dE' % i for i in range(1, 6)]
# child_column = ['B11005_002E']
# tract_columns = (income_columns + hhsize_columns + hhrace_columns +
#                 hhown_columns + work_columns + child_column)
#
# start_time = time.time()
# h_acs = c.tract_query(tract_columns, state='06', county='001',year=2018)
# end_time = time.time()
# # Calculate the elapsed time
# elapsed_time = end_time - start_time
#
# # Print the elapsed time
# print("Elapsed time:", elapsed_time, "seconds")

In [18]:
state = '06'  # FIPS code for the state
county = '001'  # FIPS code for the county
year = 2020  # ACS year you want to query from as an int. Works up to 2019

logging.info('Downloading ACS Marginals for: State = ' +
            state + ', County = ' + county + ', Year = ' + str(year))

# this a key needed to access te Census API
c = Census("285e70bec59918991430e98163a4cda2802b4f01")

    # Extract the columns we need for households
    # Column names: https://api.census.gov/data/2018/acs/acs5/variables.html
logging.info('Downloading raw household marginals...')

income_columns = ['B19001_0%02dE' % i for i in range(1, 18)]
hhsize_columns = ['B11016_0%02dE' % i for i in range(1, 17)]
hhrace_columns = ['B25006_0%02dE' % i for i in range(1, 11)]
hhown_columns = ['B25003_0%02dE' % i for i in range(1, 4)]
work_columns = ['B08202_0%02dE' % i for i in range(1, 6)]
child_column = ['B11005_002E']
tract_columns = (income_columns + hhsize_columns + hhrace_columns +
                hhown_columns + work_columns + child_column)

    # Downloads the ACS totals at the Census Tract level
h_acs = c.tract_query(tract_columns, state=state, county=county,
                          year=year)
logging.info('Raw household marginal download complete!')

    # Extract the columns we need for people
    # Column names: https://api.census.gov/data/2018/acs/acs5/variables.html

logging.info('Downloading person marginals...')
population = ['B01001_001E']
sex = ['B01001_002E', 'B01001_026E']
race = ['B02001_0%02dE' % i for i in range(1, 11)]
male_age_columns = ['B01001_0%02dE' % i for i in range(3, 26)]
female_age_columns = ['B01001_0%02dE' % i for i in range(27, 50)]
all_columns = population + sex + race + male_age_columns + female_age_columns
p_acs = c.tract_query(all_columns, state=state, county=county, year=year)
logging.info('Person marginal download complete!')

    # Create geo_cross_walk.csv.
    # Basically this shows the relationship between each tract, PUMA

logging.info('Creating geo_cross_walk file...')
    # %%
column_names = ['TRACT', 'PUMA', 'REGION']
geo_cross_walk = pd.DataFrame(columns=column_names)



In [19]:
file_path = '../tract_to_puma/2010_Census_Tract_to_2010_PUMA.txt'

# Read the text file using pandas
data = pd.read_csv(file_path, delimiter=',')


In [14]:
data

,STATEFP,COUNTYFP,TRACTCE,PUMA5CE
0,1,1,20100,2100
1,1,1,20200,2100
2,1,1,20300,2100
3,1,1,20400,2100
4,1,1,20500,2100
...,...,...,...,...
74086,78,30,960900,100
74087,78,30,961000,100
74088,78,30,961100,100
74089,78,30,961200,100


In [24]:
puma = data[(data['STATEFP'] == int(state)) & (data['COUNTYFP'] == int(county)) & (data['TRACTCE'] == int('420100'))]['PUMA5CE']

In [25]:
puma

3560    101
Name: PUMA5CE, dtype: int64

In [30]:
if year >= 2020:
    raw_xwalk_path = '../tract_to_puma/2020_Census_Tract_to_2020_PUMA.txt'
else:
    raw_xwalk_path = '../tract_to_puma/2010_Census_Tract_to_2010_PUMA.txt'

raw_xwalk = pd.read_csv(raw_xwalk_path, delimiter=',')

puma_codes_unique = []
puma_codes_all = []

for tract in h_acs['tract']:
    code = raw_xwalk[(raw_xwalk['STATEFP'] == int(state)) & (raw_xwalk['COUNTYFP'] == int(county)) & (raw_xwalk['TRACTCE'] == int(tract))]['PUMA5CE'].item()
    if code not in puma_codes_unique:
        puma_codes_unique.append(code)
    puma_codes_all.append(int(code))
h_acs['PUMA'] = puma_codes_all

102
102
102
107
107
107
108
109
108
109
109
109
109
109
108
109
109
110
110
110
102
103
102
103
103
103
103
102
102
103
107
105
106
107
107
107
107
107
110
104
104
110
102
103
103
102
102
107
108
104
101
105
109
109
110
102
103
108
108
108
109
110
110
103
103
102
102
103
102
102
102
102
102
104
102
109
108
108
109
108
108
110
110
110
110
110
110
102
102
102
102
103
103
102
102
102
104
102
102
104
102
104
104
102
102
104
104
104
103
104
101
101
101
101
101
101
103
105
106
106
105
105
105
102
102
102
103
104
104
104
108
109
101
109
109
109
108
108
110
110
110
109
107
106
107
107
108
107
108
108
108
109
109
109
110
110
110
110
110
101
101
101
110
110
110
110
110
110
110
110
110
110
101
101
103
102
102
105
102
103
102
102
103
102
102
110
110
102
102
102
106
106
102
105
105
109
108
110
110
106
106
107
107
101
101
101
105
105
106
106
105
106
105
105
105
107
107
106
106
107
107
107
107
107
108
110
105
109
108
106
107
109
105
105
105
105
105
106
105
107
107
101
105
105
106
105
105
106
106
107


In [20]:
tracts = []
tract_relationship_path = '../tract_to_puma/2020_Census_Tract_to_2010_Census_Tract.txt'
tract_relationship = pd.read_csv(tract_relationship_path, delimiter='|', dtype = {'GEOID_TRACT_20': str, 'GEOID_TRACT_10': str})

In [21]:
tract_relationship

,OID_TRACT_20,GEOID_TRACT_20,NAMELSAD_TRACT_20,AREALAND_TRACT_20,AREAWATER_TRACT_20,MTFCC_TRACT_20,FUNCSTAT_TRACT_20,OID_TRACT_10,GEOID_TRACT_10,NAMELSAD_TRACT_10,AREALAND_TRACT_10,AREAWATER_TRACT_10,MTFCC_TRACT_10,FUNCSTAT_TRACT_10,AREALAND_PART,AREAWATER_PART
0,20790540092527,01001020100,Census Tract 201,9825304,28435,G5020,S,20740540092527,01001020100,Census Tract 201,9827271,28435,G5020,S,9820448,28435
1,20790540092527,01001020100,Census Tract 201,9825304,28435,G5020,S,20740540092534,01001020200,Census Tract 202,3325674,5669,G5020,S,4856,0
2,20790540092534,01001020200,Census Tract 202,3320818,5669,G5020,S,20740540092534,01001020200,Census Tract 202,3325674,5669,G5020,S,3320818,5669
3,20790540092528,01001020300,Census Tract 203,5349271,9054,G5020,S,20740540092528,01001020300,Census Tract 203,5349271,9054,G5020,S,5349271,9054
4,20790540092529,01001020400,Census Tract 204,6384282,8408,G5020,S,20740540092529,01001020400,Census Tract 204,6384282,8408,G5020,S,6384282,8408
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126445,207904252102449,78030961100,Census Tract 9611,3479232,0,G5020,S,20740228304757,78030961200,Census Tract 9612,1017540,802134,G5020,S,14104,0
126446,20790228304757,78030961200,Census Tract 9612,1101324,802134,G5020,S,20740228304707,78030960400,Census Tract 9604,11709358,413661,G5020,S,217781,0
126447,20790228304757,78030961200,Census Tract 9612,1101324,802134,G5020,S,207404252102449,78030961100,Census Tract 9611,3513895,0,G5020,S,203,0
126448,20790228304757,78030961200,Census Tract 9612,1101324,802134,G5020,S,20740228304757,78030961200,Census Tract 9612,1017540,802134,G5020,S,883340,802134


In [15]:
old_tract = int(tract_relationship[tract_relationship['GEOID_TRACT_20'] == '06001402801'].iloc[0]['GEOID_TRACT_10'][5:])

In [16]:
old_tract

402700

In [22]:

for tract in h_acs['tract']:
    geoid_tract_20 = state + county + str(tract)
    tracts.append(int(tract_relationship[tract_relationship['GEOID_TRACT_20'] == geoid_tract_20].iloc[0]['GEOID_TRACT_10'][5:]))

In [23]:
tracts

[400100,
 400200,
 400300,
 400400,
 400500,
 400600,
 400700,
 400700,
 400900,
 401000,
 401100,
 401200,
 401300,
 401400,
 401500,
 401600,
 401700,
 401800,
 402200,
 402400,
 402500,
 402600,
 402700,
 402700,
 402800,
 402900,
 403000,
 403100,
 403300,
 403300,
 403400,
 403400,
 403501,
 403502,
 403502,
 403701,
 403600,
 403800,
 403900,
 404000,
 404101,
 404102,
 404101,
 404300,
 404400,
 404501,
 404502,
 404502,
 404700,
 404800,
 404900,
 405000,
 405100,
 405200,
 405301,
 405302,
 405401,
 405402,
 405500,
 405600,
 405700,
 405800,
 405901,
 405902,
 403300,
 406000,
 406201,
 406202,
 406300,
 406400,
 406500,
 406601,
 406602,
 406700,
 406800,
 406900,
 407000,
 430500,
 435102,
 435103,
 435103,
 435200,
 435300,
 435400,
 435500,
 435601,
 435602,
 435700,
 435800,
 433400,
 436000,
 436100,
 436200,
 436300,
 436300,
 435102,
 436401,
 436401,
 436500,
 436601,
 436602,
 436700,
 436800,
 436900,
 437000,
 435900,
 437102,
 437200,
 437300,
 437400,
 437500,
 

In [24]:
raw_xwalk_path = '../tract_to_puma/2010_Census_Tract_to_2010_PUMA.txt'
raw_xwalk = pd.read_csv(raw_xwalk_path, delimiter=',')

puma_codes_unique = []
puma_codes_all = []

for tract in tracts:
    code = raw_xwalk[(raw_xwalk['STATEFP'] == int(state)) & (raw_xwalk['COUNTYFP'] == int(county)) & (raw_xwalk['TRACTCE'] == int(tract))]['PUMA5CE'].item()
    if code not in puma_codes_unique:
        puma_codes_unique.append(code)
    puma_codes_all.append(int(code))

In [26]:
puma_codes_unique

[103, 102, 104, 106, 107, 105, 108, 109, 110, 101]